In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import pickle
import re
from tqdm import tqdm
from pathlib import Path
import urllib
import pyarrow

We will pull the Star Wars pickles to extract just the planets/locales information

In [2]:
files = sorted(Path('../WPscraped').glob('*.pickle'))
files

[WindowsPath('../WPscraped/starwars_all_canon_data_1.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_2.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_3.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_4.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_5.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_6.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_7.pickle'),
 WindowsPath('../WPscraped/starwars_all_canon_data_8.pickle')]

In [3]:
#create the dictionary to run
data = {}
for fn in files:
    with open(fn, 'rb') as f:
        part = pickle.load(f)
    data.update(part)

# The code is to open the data and to count to make sure all files in the .pickles are carried over
len(data)

39460

In [4]:
#find the key in the dictionary
def find_key(key_name, data):
    for key, value in data.items():
        if key_name == key:
            return value
        if isinstance(value, dict):
            value = find_key(key_name, value)
            if value is not None:
                return value
    return None

def get_first(key_name, data):
    result = find_key(key_name, data)
    if isinstance(result, list):
        result = result[0]
    return result

In [5]:
data.keys()

dict_keys(['%22Accu-Strike%22_integrated_targeting_computer', '%22Autoguard%22_Cybernetic_Reflex_Suite', '%22Bantha_Eye%22_laser_sight', '%22Black_Hole_Box%22_T-11_Armored_Databank', '%22Blind_Alley%22_techniques', '%22Blinder%22_4B2_jamming_array', '%22Blissex-head%22_bit-driver', '%22Breaker%22_heavy_hydrospanner', '%22Bunker_Buster%22_shell', '%22Changeling%22_Mark_71NB', '%22Community%22_junk_yard', '%22Crosshairs%22_98j_precision_holo-targeter', '%22Deadeye%22_Sight', '%22Defender%22_deflector_shield_projector', '%22Finisher%22_Rocket_Rack', '%22Firecaller%22_light_flame_projector', '%22Gundark%22_scav-suit', '%22Huttsplitter%22_blaster_rifle', '%22Impact%22_repulsor_cannon', '%22Jakku_Night_Special%22_blaster_rifle', 'Targeter_System', '%22Mercy%22_grenade', '%22Multi_Imager%22_dedicated_energy_receptor', '%22Muse%22', '%22My_First_Blaster%22_Safety_Kit', '%22Outreach%22_Target_Sensor', '%22Peacekeeper%22_stun_baton', '%22Personal_Armory%22_Gear', '%22Precision-X%22_Marksman_Rifl

In [6]:
print(data['Ilum'])

{'url': 'https://starwars.fandom.com/wiki/Ilum', 'title': '\n\t\t\t\t\tIlum\t\t\t\t', 'is_character': False, 'side_bar': {'Astrographical information': {'Region': 'Unknown Regions', 'Sector': '7G sector', 'System': ['Ilum system', 'Starkiller Base system'], 'Suns': ['Asar', 'Unidentified star'], 'Orbital position': '5', 'Moons': ['2', '0'], 'Grid square': ['G-7'], 'Trade routes': 'Metellos-Ilum hyperspace route', 'Rotation period': '66 hours', 'Orbital period': ['301 days', 'Variable']}, 'Physical information': {'Diameter': '660 kilometers', 'Atmosphere': ['Breathable'], 'Climate': 'Frigid', 'Primary terrain': ['Crystallized glaciers', 'Ice plains', 'Mountains', 'Forests'], 'Points of interest': ['Jedi Temple', 'Crystal Cave', 'Imperial Trench', 'Starkiller Base'], 'Flora': 'Trees', 'Fauna': ['Asharl panther', 'Blismal', 'Gorgodon', 'Razhak', 'Snowfeather bird', 'Unidentified creature']}, 'Societal information': {'Immigrated species': 'Human', 'Population': ['5', '200'], 'Major exports

In [9]:
print(data['Corellia'])

{'url': 'https://starwars.fandom.com/wiki/Corellia', 'title': '\n\t\t\t\t\tCorellia\t\t\t\t', 'is_character': False, 'side_bar': {'Astrographical information': {'Region': 'Core Worlds', 'Sector': 'Corellian sector', 'System': 'Corellia system', 'Suns': ['1:', 'Corell'], 'Moons': ['Multiple:', "Corellia's nearest moon", 'Gus', 'Gus Talon'], 'Grid square': 'M-11', 'Trade routes': ['Corellian Run', 'Corellian Trade Spine'], 'Rotation period': '25 standard hours', 'Orbital period': '329 standard days'}, 'Physical information': {'Class': 'Terrestrial', 'Diameter': ['11', '000 kilometers'], 'Atmosphere': 'Breathable', 'Climate': 'Temperate', 'Primary terrain': ['Forests', 'Jungles', 'Oceans', 'Industrial urban'], 'Points of interest': ['Bottoms', 'Gilded Descent Casino', 'Imperial training center', 'Navigation institute', 'Santhe Shipyards'], 'Flora': 'Tree', 'Fauna': ['Bluevev glider', 'Coppergrin', 'Corellian hound', 'Creedok', 'Corellian sand panther', 'Fingerlip garpon', 'Fleek eel', 'Ga

In [12]:
print(data['Coruscant'])

{'url': 'https://starwars.fandom.com/wiki/Coruscant', 'title': '\n\t\t\t\t\tCoruscant\t\t\t\t', 'is_character': False, 'side_bar': {'Astrographical information': {'Region': ['Core Worlds', 'The Interior'], 'Sector': ['Corusca sector', 'Coruscant subsector'], 'System': 'Coruscant system', 'Suns': '1: Coruscant Prime', 'Moons': ['4', 'Centax-1', 'Centax-2', 'Centax-3', 'Hesperidium'], 'Grid square': 'L-9', 'XYZ coordinates': ['0', '0', '0'], 'Trade routes': ['Corellian Run', 'Koros Trunk Line', 'Metellos Trade Route', 'Namadii Corridor', 'Nexus Route', 'Perlemian Trade Route'], 'Rotation period': '24 standard hours', 'Orbital period': '365 standard days'}, 'Physical information': {'Diameter': ['12', '240 kilometers'], 'Atmosphere': 'Breathable', 'Climate': 'Temperate ', 'Primary terrain': ['Ecumenopolis', 'Dense', 'planetwide multilevel urban city'], 'Points of interest': ['Bureau of Ships and Services Heritage Museum', 'CoCo Town', 'Coruscant spaceport', 'Coruscant Imperial shipyard', '

In [13]:
print(data['AT-KT'])

{'url': 'https://starwars.fandom.com/wiki/AT-KT', 'title': '\n\t\t\t\t\tAT-KT\t\t\t\t', 'is_character': False, 'side_bar': {'Production information': {'Line': 'All-terrain vehicle', 'Model': 'All Terrain Scout Transport', 'Class': 'Walker'}, 'Technical specifications': {'Armament': 'Stun weaponry'}, 'Usage': {'Affiliation': ['Galactic Empire', 'Imperial Army']}}, 'paragraph': '\nThe AT-KT, nicknamed the "Hunter," was a less common variant of the Galactic Empire\'s All Terrain Scout Transport. The AT-KT was widely used on the forest planet Kashyyyk during the Imperial occupation, and was used to subjugate the Wookiees. Its weaponry was created to stun mobile targets, in order to then enslave them.\n', 'crosslinks': ['Wookiee', 'All_Terrain_Attack_Pod', 'All_Terrain_Combat_Excavator', 'All_Terrain_Open_Transport', 'All_Terrain_Missile_Platform', 'AT-DP_Arc_Cannon_Prototype', 'Slavery', 'All_Terrain_Patrol_Droid', 'All_Terrain_Scout_Transport', 'Walker', 'Planet', 'AT-ST_Mark_III', 'All_T